# 🔄 CNN Advanced Resume Notebook (без втрат)

Цей ноутбук продовжує перерване навчання **CNN Advanced** з `weights/advanced_last.pth` і зберігає попередню історію епох.


## 1) Налаштування середовища (Colab)


In [ ]:
# Якщо працюєш у Colab
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
print("📦 Installing dependencies...")
!pip install -q torch torchvision torchaudio pandas numpy tqdm matplotlib
print("✅ Dependencies installed")


In [ ]:
import os
import json
import shutil
from datetime import datetime

import torch
import matplotlib.pyplot as plt

print("🔥 CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


## 2) Шляхи та перевірка файлів


In [ ]:
# Зміни шлях під свій Drive
PROJECT_PATH = '/content/drive/MyDrive/diplom/backend'
os.chdir(PROJECT_PATH)
print("📂 Working directory:", os.getcwd())

required_files = ['train.py', 'dataset.py', 'models/__init__.py', 'data/sudoku.csv']
for file in required_files:
    print(f"✅ {file}" if os.path.exists(file) else f"❌ {file} - NOT FOUND")

os.makedirs('weights', exist_ok=True)


In [ ]:
# 🔍 ПЕРЕВІРКА ВЕРСІЇ train.py (має бути БЕЗ eager imports)
print("🔍 Checking train.py version...")
with open('train.py', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    # Перевіряємо рядки 16-18
    has_old_import = any('from models import CNNBaseline, CNNAdvanced, GNNModel, SudokuRNN' in line for line in lines[15:20])
    
    if has_old_import:
        print("❌ СТАРА версія train.py виявлена!")
        print("   Файл має eager import всіх моделей, що викликає torch_geometric помилку.")
        print("   ")
        print("   РІШЕННЯ: Синхронізуй оновлений train.py з локальної машини в Google Drive,")
        print("            або перезапусти Runtime: Runtime → Restart runtime")
        raise RuntimeError("train.py потребує оновлення! Див. COLAB_SYNC_INSTRUCTIONS.md")
    else:
        print("✅ train.py версія коректна (lazy imports)")
        print("   Моделі завантажуються тільки коли потрібні - torch_geometric не потрібен для CNN Advanced")


## 3) Resume-конфігурація

> `total_epochs` — це **загальна ціль** навчання (наприклад 40), а не "ще докинути N епох".


In [ ]:
# === СТАНДАРТНІ ПАРАМЕТРИ CNN ADVANCED ===
TOTAL_EPOCHS = 40
BATCH_SIZE = 64
LR = 0.001
WEIGHT_DECAY = 1e-5
HIDDEN_CHANNELS = 128
NUM_RESIDUAL_BLOCKS = 20
NUM_WORKERS = 2
LIMIT = 1000000
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

CKPT_LAST = 'weights/advanced_last.pth'
CKPT_BEST = 'weights/advanced_best.pth'
HISTORY_PATH = 'weights/advanced_history.json'
HISTORY_BACKUP = f"weights/advanced_history_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

print('Device:', DEVICE)
print('Checkpoint last exists:', os.path.exists(CKPT_LAST))
print('History exists:', os.path.exists(HISTORY_PATH))


In [ ]:
# Зчитуємо стан перед запуском
initial_history = []
if os.path.exists(HISTORY_PATH):
    with open(HISTORY_PATH, 'r') as f:
        initial_history = json.load(f)
    shutil.copy2(HISTORY_PATH, HISTORY_BACKUP)
    print(f"🛟 Backup history: {HISTORY_BACKUP}")

start_epoch = 1
resume_mismatch_warning = None
if os.path.exists(CKPT_LAST):
    try:
        ckpt = torch.load(CKPT_LAST, map_location='cpu')
    except Exception as e:
        print(f"⚠️ Standard torch.load failed: {e}")
        print("Retrying with weights_only=False for trusted local checkpoint...")
        ckpt = torch.load(CKPT_LAST, map_location='cpu', weights_only=False)

    start_epoch = int(ckpt['epoch']) + 1
    print(f"🔄 Resume from epoch {start_epoch} (last finished: {ckpt['epoch']})")

    ckpt_args = ckpt.get('args', {}) if isinstance(ckpt, dict) else {}
    expected = {
        'model': 'advanced',
        'hidden_channels': HIDDEN_CHANNELS,
        'num_residual_blocks': NUM_RESIDUAL_BLOCKS,
    }
    mismatches = []
    for k, v in expected.items():
        if k in ckpt_args and ckpt_args[k] != v:
            mismatches.append(f"{k}: ckpt={ckpt_args[k]} != current={v}")

    if mismatches:
        resume_mismatch_warning = "; ".join(mismatches)
        print("⚠️ Checkpoint config mismatch detected:")
        for item in mismatches:
            print("   -", item)
        print("   Training may fail on --resume. Consider deleting CKPT_LAST or changing params.")
else:
    print("ℹ️ Checkpoint not found: training from scratch")

if start_epoch > TOTAL_EPOCHS:
    print(f"✅ Навчання вже завершено: checkpoint має epoch {start_epoch - 1}, ціль {TOTAL_EPOCHS}")


## 4) Запуск навчання (resume або fresh)


In [ ]:
# Перевірка чи потрібно навчання
if start_epoch > TOTAL_EPOCHS:
    print(f"⏭️ Training already completed (epoch {start_epoch - 1}/{TOTAL_EPOCHS})")
    skip_training = True
else:
    skip_training = False
    print(f"🚀 Will train epochs {start_epoch} to {TOTAL_EPOCHS}")
    if os.path.exists(CKPT_LAST):
        print(f"   Resume: Yes (from checkpoint)")
    else:
        print(f"   Resume: No (fresh start)")


In [ ]:
# === НАВЧАННЯ CNN ADVANCED (RESUME) ===
# Використовує ! для real-time виводу в Colab

if not skip_training:
    !python -u train.py \
        --model advanced \
        --data data/sudoku.csv \
        --epochs {TOTAL_EPOCHS} \
        --batch-size {BATCH_SIZE} \
        --lr {LR} \
        --weight-decay {WEIGHT_DECAY} \
        --hidden-channels {HIDDEN_CHANNELS} \
        --num-residual-blocks {NUM_RESIDUAL_BLOCKS} \
        --num-workers {NUM_WORKERS} \
        --limit {LIMIT} \
        --save-dir weights \
        --device {DEVICE} \
        --resume {CKPT_LAST if os.path.exists(CKPT_LAST) else ''}


## 5) Відновлення ПОВНОЇ історії епох (щоб не втратити ранні епохи)


In [ ]:
# train.py при resume перезаписує history (не дописує).
# Тут ми зливаємо стару + нову історію по номеру epoch.

merged = {}
for item in initial_history:
    merged[int(item['epoch'])] = item

if os.path.exists(HISTORY_PATH):
    with open(HISTORY_PATH, 'r') as f:
        new_history = json.load(f)
    for item in new_history:
        merged[int(item['epoch'])] = item

full_history = [merged[k] for k in sorted(merged.keys())]

with open(HISTORY_PATH, 'w') as f:
    json.dump(full_history, f, indent=2)

print(f"✅ Full history epochs: {len(full_history)}")
if full_history:
    print(f"Epoch range: {full_history[0]['epoch']}..{full_history[-1]['epoch']}")


## 6) Графіки та фінальні метрики


In [ ]:
with open(HISTORY_PATH, 'r') as f:
    history = json.load(f)

if not history:
    raise ValueError("History is empty. Check training run / checkpoints.")

epochs = [h['epoch'] for h in history]
train_loss = [h['train']['loss'] for h in history]
val_loss = [h['val']['loss'] for h in history]
val_acc = [h['val']['cell_accuracy'] for h in history]
board_acc = [h['val']['board_accuracy'] for h in history]
lr_hist = [h['lr'] for h in history]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].plot(epochs, train_loss, label='Train Loss')
axes[0].plot(epochs, val_loss, label='Val Loss')
axes[0].set_title('Loss')
axes[0].grid(True)
axes[0].legend()

axes[1].plot(epochs, val_acc, label='Val Cell Accuracy', color='tab:blue')
axes[1].plot(epochs, board_acc, label='Val Board Accuracy', color='tab:green')
axes[1].set_title('Accuracy')
axes[1].grid(True)
axes[1].legend()

axes[2].plot(epochs, lr_hist, label='LR', color='tab:red')
axes[2].set_title('Learning rate')
axes[2].set_yscale('log')
axes[2].grid(True)
axes[2].legend()

plt.tight_layout()
plt.savefig('weights/advanced_resume_curves.png', dpi=250, bbox_inches='tight')
plt.show()

print('\n📊 FINAL SUMMARY')
print('Best Val Loss:      ', min(val_loss))
print('Best Cell Accuracy: ', max(val_acc))
print('Best Board Accuracy:', max(board_acc))
print('Last Epoch:         ', epochs[-1])
print('Saved plot: weights/advanced_resume_curves.png')
